# Header

This ipynb was written by Daniel Kobko for Assignment 3 of CIS*4780.
The last modified date is 4/4/2022. Purpose: To classify the CIFAR-10 dataset
using a CNN, and FCNN.

# Imports

In [73]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

from torchvision import datasets, transforms
import torchvision.utils

import numpy as np

# General

In [74]:
# General Hyperparameters
num_classes = 10
num_epochs = 5
batch_size = 10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')
mean = 0.5
means = (mean, mean, mean)
dev = 0.2
std = (dev, dev, dev)

In [75]:
# FCNN Hyperparameters
FCNNinput = 3072 # input size
FCNNLR = 0.0004 # learning rate

In [76]:
# CNN Hyperparameters
CNNinput = 2500 # input size
CNNLR = 0.0004 # learning rate
CNNkernel = 5 # kernel size

#Load in the CIFAR-10 dataset

In [77]:
transforms = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize(mean, std)
    ]
    )

train = datasets.CIFAR10(root='./data', train=True, 
                         transform=transforms, 
                         download=True)
test = datasets.CIFAR10(root='./data', train=False,
                        transform=transforms,
                        download=True)

# 50,000 training images split to include validation
train, valid = random_split(train, [40000,10000])

trainloader = DataLoader(train, batch_size=batch_size)
validloader = DataLoader(valid, batch_size=batch_size)
testloader = DataLoader(test, batch_size=batch_size)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Networks - Train, Validate, Test sets

Fully Connected

In [78]:
class FCNNetwork(nn.Module):
  def __init__(self):
    super(FCNNetwork, self).__init__()

    self.input_layer = nn.Linear(FCNNinput, 1000)
    self.fc1 = nn.Linear(1000, 500)
    self.fc2 = nn.Linear(500, 100)
    self.output_layer = nn.Linear(100, num_classes)

  def forward(self, x):
    x = F.relu(self.input_layer(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.output_layer(x)
    return x

In [79]:
# Initialize the network
FCNN = FCNNetwork()
if torch.cuda.is_available():
    FCNN = FCNN.cuda()

# Define the loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(FCNN.parameters(), lr=FCNNLR)

In [80]:
min_valid_loss = np.inf # keep track of the lowest loss

for e in range(num_epochs):
  # Training Set
  training_loss = 0.0
  for data, labels in trainloader:
    # Switch to GPU if available
    if torch.cuda.is_available():
        data, labels = data.cuda(), labels.cuda()
    optimizer.zero_grad()
    data = data.reshape(data.shape[0], -1)
    target = FCNN(data)
    loss = loss_func(target, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
  
  # Validation Set
  validation_loss = 0.0
  FCNN.eval()
  for data, labels in validloader:
    # Switch to GPU if available
    if torch.cuda.is_available():
        data, labels = data.cuda(), labels.cuda()
    data = data.reshape(data.shape[0], -1)
    target = FCNN(data)
    loss = loss_func(target,labels)
    validation_loss += loss.item()

  # Output Results
  print('------------------------')
  print(f'Epoch {e+1}')
  print('------------------------')
  print(f'Training loss: {training_loss / len(trainloader)}')
  print(f'Validation loss: {validation_loss / len(validloader)}')
  
  # Check for if the current state is worth saving
  if min_valid_loss > validation_loss:
    improvement = min_valid_loss - validation_loss
    print(f'Validation loss improved by {improvement:.6f}. Saving The Model')
    min_valid_loss = validation_loss

    PATH = 'saved_model.pth'
    torch.save(FCNN.state_dict(), PATH)
  else:
    print('Validation loss was not improved.')

------------------------
Epoch 1
------------------------
Training loss: 1.705490978434682
Validation loss: 1.565824520111084
Validation loss improved by inf. Saving The Model
------------------------
Epoch 2
------------------------
Training loss: 1.483692170098424
Validation loss: 1.4837581121325494
Validation loss improved by 82.066408. Saving The Model
------------------------
Epoch 3
------------------------
Training loss: 1.3548876774460077
Validation loss: 1.4809493918418883
Validation loss improved by 2.808720. Saving The Model
------------------------
Epoch 4
------------------------
Training loss: 1.241989163838327
Validation loss: 1.4943923203349114
Validation loss was not improved.
------------------------
Epoch 5
------------------------
Training loss: 1.1377356410697101
Validation loss: 1.5345469516813754
Validation loss was not improved.


In [81]:
dataiter = iter(testloader)
images, labels = dataiter.next()

FCNN = FCNNetwork()
FCNN.load_state_dict(torch.load(PATH))
images = images.reshape(images.shape[0], -1)
outputs = FCNN(images)
_, predicted = torch.max(outputs, 1)

correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
data_loss = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.reshape(images.shape[0], -1)
        outputs = FCNN(images)
        loss = loss_func(outputs,labels)
        data_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Overall testing accuracy: {100 * correct // total}%')
print(f'Loss: {loss.item()}')

Overall testing accuracy: 49%
Loss: 1.619200348854065


Convolutional

In [82]:
# Define the network

In [83]:
class CNNetwork(nn.Module):
  def __init__(self):
      super(CNNetwork,self).__init__()
      
      self.conv1 = nn.Conv2d(3, 75, CNNkernel)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(75, 100, CNNkernel)

      self.inputlayer = nn.Linear(CNNinput, 1000)
      self.fc1 = nn.Linear(1000, 500)
      self.fc2 = nn.Linear(500, 250)
      self.fc3 = nn.Linear(250, 100)
      self.fc4 = nn.Linear(100, 50)
      self.fc5 = nn.Linear(50, 20)
      self.outputlayer = nn.Linear(20, num_classes)
      
  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = torch.flatten(x, 1) # flatten all dimensions except batch
      x = F.relu(self.inputlayer(x))
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = F.relu(self.fc4(x))
      x = F.relu(self.fc5(x))
      x = self.outputlayer(x)
      return x

In [84]:
CNN = CNNetwork()

if torch.cuda.is_available():
    CNN = CNN.cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNN.parameters(), lr = CNNLR)

In [85]:
# Training steps

In [86]:
# keeps track of state with the lowest loss
min_valid_loss = np.inf

for e in range(num_epochs):
    train_loss = 0.0
    for data, labels in trainloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        train_data = CNN(data)
        loss = loss_func(train_data, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    valid_loss = 0.0
    CNN.eval()
    for data, labels in validloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        valid_data = CNN(data)
        loss = loss_func(valid_data,labels)
        valid_loss += loss.item()
    print('------------------------')
    print(f'Epoch {e+1}')
    print('------------------------')
    print(f'Training loss: {train_loss / len(trainloader)}')
    print(f'Validation loss: {valid_loss / len(validloader)}')
     
    if min_valid_loss > valid_loss:
        improvement = min_valid_loss - valid_loss
        print(f'Validation loss improved by {improvement:.6f}. Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        PATH = 'saved_model.pth'
        torch.save(CNN.state_dict(), PATH)
    else:
        print('Validation loss was not improved.')

------------------------
Epoch 1
------------------------
Training loss: 1.7325327844023704
Validation loss: 1.4440854900479316
Validation loss improved by inf. Saving The Model
------------------------
Epoch 2
------------------------
Training loss: 1.255355682320893
Validation loss: 1.1752740270495414
Validation loss improved by 268.811463. Saving The Model
------------------------
Epoch 3
------------------------
Training loss: 1.0017531302068383
Validation loss: 1.0959053260982037
Validation loss improved by 79.368701. Saving The Model
------------------------
Epoch 4
------------------------
Training loss: 0.8115438650753349
Validation loss: 1.041911537066102
Validation loss improved by 53.993789. Saving The Model
------------------------
Epoch 5
------------------------
Training loss: 0.6476335989334621
Validation loss: 1.198700613953173
Validation loss was not improved.


In [87]:
# Test the network

In [88]:
dataiter = iter(testloader)
images, labels = dataiter.next()

CNN = CNNetwork()
CNN.load_state_dict(torch.load(PATH))
outputs = CNN(images)
_, predicted = torch.max(outputs, 1)

correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

data_loss = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = CNN(images)
        loss = loss_func(outputs,labels)
        data_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Overall testing accuracy: {100 * correct // total}%')
print(f'Loss: {data_loss/len(testloader)}')

Overall testing accuracy: 66%
Loss: 1.0341052894219755
